Implementing an YOLO Algorithm is highly complex and very time consuming. In this assignment, I will walk you through the process. 
The data set I used for the assignment is in this [link ](https://storage.googleapis.com/open_source_datasets/ShelfImages.tar.gz)
The data set is split into two parts namely train and test

This dataset contains only images but not the annotations
Annotations to this file are found in a text file in github. The link to the github repository consisting annotations.txt file  is [here ](https://github.com/gulvarol/grocerydataset/)

In any code cell, Uncomment the Print commands to Understand the output. And the data flow. 

# **For Data Preperation**

It is very complicated in google colab to download or upload files. This would be helpful

In [0]:
# Defining Functions that are useful in  future to download , Upload and print files.
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# use this to upload files
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# use this to download a file  
def download(path):
  from google.colab import files
  files.download(path)

**Downloading Dataset**

In [0]:
#this code sets the data up, the extracted file shelfimages can probably be shared as shelfimages1 ev

!git clone https://github.com/gulvarol/grocerydataset/ #cloning git repository of meta data and description

#!wget https://github.com/gulvarol/grocerydataset/releases/download/1.0/GroceryDataset_part1.tar.gz #downloading part1 data
#!wget https://github.com/gulvarol/grocerydataset/releases/download/1.0/GroceryDataset_part2.tar.gz #downloading part2 data
#!tar --gunzip --extract --verbose --file=GroceryDataset_part1.tar.gz #extracting the tar files into a folder
#!tar --gunzip --extract --verbose --file=GroceryDataset_part2.tar.gz 

!wget https://storage.googleapis.com/open_source_datasets/ShelfImages.tar.gz #downloads add on provided by the assignment
 
!tar --gunzip --extract --verbose --file=ShelfImages.tar.gz #extracts add ondata tar folder

**Data Preperation**

In [0]:
#changing working directory 
%cd grocerydataset/

In [0]:
#reading the data from an annotation file to a dictionary with file names as key and the data of the objects in it as a dict (list ) attached to the key


#open a file into a file variable f 
f = open('annotation.txt') 

#Loads the entire content into a variable l ( A large string ). Uncomment print(l) to find more
l = f.read()
#print(l)

#The following command Splits the big text and writes each line into a fresh string. And compiles the strings into an array of strings uncomment to witness 
k = l.splitlines()
m = list(k)
#print(m)

#creating an empty dict 
Dictionarydata = dict()

#Iterating over list of strings. Each i refers to one image 
for i in m :
  #splits each string into various little strings whenever a space is encountered
  p = i.split()
  #the first string in that contains the name of image. So we create a dict with name of the image as key  and write the rest of the data to that key as values uncomment printbelow to find it out
  Dictionarydata[p[0]]= p[1:]
  #print(Dictionarydata)

  


In [0]:
#taking the list of files in test and train directories and storing them as a list of strings. 
#os.lisdir() Function lists all the available files in the current working directory into a list of strings. Uncomment print to understand the output
import os 
%cd ..
%cd 'ShelfImages/test'
testlist = os.listdir()
#print(testlist)
%cd ..
%cd train
trainlist = os.listdir()
#print(trainlist)
%cd ..


So, as we described in the Document, We have a list of all the train and test image names.

In [0]:
#yolo usually asks for text files corresponding to each image
#In order to create names for text files this would be very handy 


#creating two empty lists
trainlistnamestextfiles = list() 
testlistnamestextfiles = list()

#iterating over the Previously obtained lists which contains the 
for i in trainlist :
  #the last three letters of image names are jpg when we remove them and name txt , we have the names of the corresponding text files uncomment print function in the end to find out how they unveil 
  trainlistnamestextfiles.append(str(i[:-3])+'txt')
for i in testlist :
  testlistnamestextfiles.append(str(i[:-3])+'txt')

#print(trainlistnamestextfiles)
#print(testlistnamestextfiles)  

In [0]:
import numpy as np
import cv2 
%cd 
%cd ..
%cd /content/ShelfImages/train

#Now we are iterating over the trainlist and testlist Lists so that we can do data processing

for i in range(len(trainlist)) :

  #as we know that the dictionarydata contains the dict with images names as key and A long list of strings which are numbers as the value
  arrayofsinglefile = Dictionarydata[trainlist[i]] 
  #print(arrayofsinglefile, '\n')

  # The First string of the list is the number of detections obtained in a particular image 
  countofdetectionsinparticularimage = arrayofsinglefile[0]
  #print(countofdetectionsinparticularimage)
  #print('\n')

  #The rest of the list is x, y , width , height , class repeated count number of times. so Now we reshape the list and write each object in a row and 5 columns containing , 
  array = np.array(arrayofsinglefile[1:],dtype = float).reshape(int(countofdetectionsinparticularimage),5)
  
  #now we need classes in the first column according to the format YOLO accepts. and all other colums should be moved right. So we use np.roll() 
  #This pushes 4 all the colums to one step right and brings the lost column to the first

  array=np.roll(array, 1, axis=1)
  #print(array, '\n')

  #as I said that the inputs of each files to YOLO should be floats
  image = cv2.imread(trainlist[i])
  height,width= image.shape[0],image.shape[1]
  #print(height,width)
  #print('\n')
  #print(len(array))
  #print(array[:][:])
  for j in range(int(countofdetectionsinparticularimage)):
    for k in range(1,5):
      #print(k)
      if(k== 2):
        array[j][k]=int(array[j][k])/height
        #if array[j][k]>1:
          #print('error2')
          #print(array[j][k],height,width) # uncomment these if you find an error stating that , the values of weights either less than zero or greater than 1 
      elif(k == 4):
        array[j][k]=int(array[j][k])/height
        #if array[j][k]>1: 
          #print('error4')
          #print(array[j][k],height,width)
      elif(k== 3):
        array[j][k]=int(array[j][k])/width
        #if array[j][k]>1:
          #print('error3')
          #print(array[j][k],height,width)
      elif(k==1 ):
        array[j][k]=int(array[j][k])/width
        #if array[j][k]>1:
          #print('error1')
          #print(array[j][k],height,width)
  #print(array, '\n')



  # thus we obtained an array that is exactly same as how each file should look 
  #opening the file and writing the array in the text file
  
  with open('{}'.format(trainlistnamestextfiles[i]), 'w') as f:
    for x in range(int(countofdetectionsinparticularimage)):

      for y in range(5):
        if y == 0 :
          f.write(str(0) + ' ')
          #f.write(str(float(array[x][y])) + ' ') uncomment this for 11 objects training
        else:
          f.write(str(float(array[x][y])) + ' ')
      f.write('\n')      

In [0]:
#the comments are same as the above cell, just the folder changes. 
import numpy as np
import cv2 
%cd /content/ShelfImages/test

for i in range(len(testlist)) :
  arrayofsinglefile = Dictionarydata[testlist[i]]
  #print(arrayofsinglefile, '\n')
  countofdetectionsinparticularimage = arrayofsinglefile[0]
  #print(countofdetectionsinparticularimage)
  #print('\n')
  array = np.array(arrayofsinglefile[1:],dtype = float).reshape(int(countofdetectionsinparticularimage),5)
  # reshapethedata=array.reshape(int(countofdetectionsinparticularimage),5))
  #To swap columns use the followng code  
  #array[:,[0,4]]=array[:,[4,0]]
  #To shift each column in an array use np.roll() in numpy
  array=np.roll(array, 1, axis=1)
  #print(array, '\n')
  image = cv2.imread(testlist[i])
  height,width= image.shape[0],image.shape[1]
  #print(height,width)
  #print('\n')
  #print(len(array))
  #print(array[:][:])
  for j in range(int(countofdetectionsinparticularimage)):
    for k in range(1,5):
      #print(k)
      if(k== 2):
        array[j][k]=int(array[j][k])/height
        #if array[j][k]>1:
         # print('error2')
         # print(array[j][k],height,width)
      if(k == 4):
        array[j][k]=int(array[j][k])/height
        #if array[j][k]>1:
          #print('error4')
          #print(array[j][k],height,width)
      if(k== 3):
        array[j][k]=int(array[j][k])/width
        #if array[j][k]>1:
          #print('error3')
          #print(array[j][k],height,width)
      elif(k==1 ):
        array[j][k]=int(array[j][k])/width
        #if array[j][k]>1:
          #print('error1')
          #print(array[j][k],height,width)
  #print(array, '\n')
  with open('{}'.format(testlistnamestextfiles[i]), 'w') as f:
    for x in range(int(countofdetectionsinparticularimage)):

      for y in range(5):
        if y==0 :
          f.write(str(0) + ' ')
        else :
          f.write(str(float(array[x][y])) + ' ')
      f.write('\n')      


In [0]:
#Zipping the entire data folder and dowloading to avoid multiple data preperation. Please follow the steps below.
!zip -r /content/obj.zip /content/ShelfImages/

Here download the zip file into the folder and start performing the following operations ( you will be recquired  to run the 1,2,3 code blocks in your terminal before this)

1) Extract the content of the obj.zip file. 

2) Navigate to the extracted zone , open content/darknet/shelfimages/train 

3) copy everything in the folder

4) Navigate to root directory where darknet is cloned and made and further navigate to ./darknet/data/obj (create an obj empty file if necessary , also delete all the info in data folder before this step ) . in obj folder paste the image and text files. 

5) now in the terminal change the root directory to darknet 

6) clone the repo https://github.com/theAIGuysCode/YoloGenerateTrainingFile

7) openn the below python file , find the word jpg and change it to JPG 
now in terminal type the following 
python YoloGenerateTrainingFile/generate_train.py

8) this generates a text file train.txt. Copy the file and upload it into your google drive at yolov3 folder 

9) Now replace the contents in data/obj with contents from content/darknet/shelfimages/test 

10) repeat secondpart of  command 7

11)rename the new txt file to test.txt , copy it and upload to drive at yolov3 folder

12) open text editor write 11 object names , as you wish , one in a row , no spaces no tabs and save the file as obj.names

13) open text editor and type as follows and save it as obj.data

classes = 11

train = data/train.txt

valid = data/test.txt

names = data/obj.names

backup = /mydrive/yolov3/backup/

14) upload objnames and obj.data to drive/yolov3 

15) copy all the content from train and test folders and paste it in data/obj . compress the obj file to obj.zip and upload obj.zip to google drive/yolov3

16) Download a Yolov3.cfg file from darknet/cfg into your desktop, Edit the following 

For 1 class training:
>>
1) Search for random flag and set random= 1
>>
2) search for batch and change batch = 64
>>
3) change line subdivisions to subdivisions=16
>>
4) change line max_batches to 
max_batches =6000
>>
5) change line steps = 4800,5400 
>>
6) find classes = 80 and change it to classes = 1
>>
7)change [filters=255] to filters=18 in the 3 [convolutional] before each [yolo] layer, keep in mind that it only has to be the last [convolutional] before each of the [yolo] layers.
>>
8) Save the config file as yolov3_custom.cfg

for 11 clasess training : 
>>
1) Search for random flag and set it to 1
>>
2) search for batch and change batch = 64
>>
3) change line subdivisions to subdivisions=16
>>
4) change line max_batches to 
max_batches = 22000
>>
5) Change line steps to steps = 17600, 19800
>>
6) Change line classes = 11
>>
7)change [filters=255] to filters= 48 in the 3 [convolutional] before each [yolo] layer, keep in mind that it only has to be the last [convolutional] before each of the [yolo] layers
>>
8) Save the config file as yolov3_custom.cfg

14) Download the custiom weights from the following link [ download link](https://pjreddie.com/media/files/darknet53.conv.74)

15 ) Add all the above to the YOLOV3 folder and upload it to google drive.



Note : The anchor box ratio is just an approximation and should be tuned by observing the ratios of width and height from the dataset, Yolo original datasets usually perform well for this

# **Thus the data preperation is done and we enter into training**

Follow these steps to start training


In [0]:
#Mounting Google drive to colab
%cd 
%cd ..
%cd content/
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
# When we copy path of drive it has a space in it and is very irritating to use in many cases . Also generates some errors. So we create this link  
#The link makes the system understand that both are same. 
%cd 
%cd ..
!ln -s /content/gdrive/'My Drive/' /mydrive

In [0]:
#changing directory to content . 
%cd 
%cd ..
%cd content
# clone darknet repo
!git clone https://github.com/AlexeyAB/darknet

In [0]:
# use this code to check the cuda version installed in the GPU ( not much necessary)
!nvcc --version 

In [0]:
#Making Darknet as the working directory.
%cd darknet

# since we are GPU powered we have to change some lines of the make file to enable GPU usage on darknet 
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

In [0]:
# this makes the darknet . in windows it creates an exe software and in linux it makes 
!make

In [0]:
#list files in yolov3 folder 
!ls /mydrive/yolov3

In [0]:
#Run this cell if your data is in obj folder in yolov3 folder in your drive. 
%cd 
%cd ..
%cd content/gdrive/'My Drive'/yolov3
!zip -r /content/obj.zip ./obj

In [0]:
# unzip the zip file and its contents Into darknet/data folder.  the images and text files  should now be in /darknet/data/obj
%cd 
%cd .. 
%cd content
!unzip /content/obj.zip -d darknet/data 

In [0]:
#Copy config file Which is 

%cd 
%cd .. 
%cd content
!cp /mydrive/yolov3/yolov3_custom.cfg darknet/cfg/

In [0]:
# upload the obj.names and obj.data files and pre trained weights to cloud VM from Google Drive
%cd 
%cd .. 
%cd content
!cp /mydrive/yolov3/obj.names darknet/data
!cp /mydrive/yolov3/obj.data  darknet/data
!cp /mydrive/yolov3/darknet53.conv74  darknet #%cd darknet  #!wget https://pjreddie.com/media/files/darknet53.conv.74 would do the same if you dont want to download. 


Training for the first time

In [0]:
#This snippet of code starts training by making the 
%cd
%cd ..
%cd content/darknet
!./darknet detector train data/obj.data cfg/yolov3_custom.cfg darknet53.conv.74 -dont_show

Training after 100 iterations, if it is stopped due to connectivity issues or you find your model inaccurate and you want to train more

In [0]:
#This snipped obtains the last saved back up files and resumes training from there
%cd 
%cd ..
%cd content
!cp /content/gdrive/'My Drive'/yolov3/yolov3_custom_last.weights /darknet
%cd darknet/
!./darknet detector train data/obj.data cfg/yolov3_custom.cfg yolov3_custom_last.weights -dont_show

When to stop ? When the output is being printed , always observe the second number from the iteration number. It means avg_error. This should be near to zero. Try to stop when it is less than 1

**Prediction , Validation and Results**

to obtain a file with prediction  run the following command . Remember ? we gave valid = test.txt in obj.data. The following command reads those images and predicts over them. Then writes all the data to a file in darknet/results folder.

In [0]:
!./darknet detector valid data/obj.data cfg/yolov3_custom.cfg yolov3_custom_last.weights

Observed the new file in results folder ? Lets analyse it and return the Json format as asked. 

In [0]:
%cd results/
import numpy as np
from collections import defaultdict
res=[]
img=[]
imgprob=[]

with open('comp4_det_test_Object.txt','r') as f:
    for i in f:
        res.append(i.split())
        
for i in range(len(res)):
    img.append(res[i][0])
    if(float(res[i][1]) > 0.3):
        imgprob.append(1)
    else:
        imgprob.append(0)


d = defaultdict(list)
for k,v in zip(img, imgprob):
    d[k].append(v)
 
for i in d.keys():
    lis=d[i]
    d[i]=sum(lis)
#print(d)

import json
print(json.dumps(d, sort_keys=True, indent=4))
    
with open('image2products.json', 'w') as fp:
    json.dump(d, fp)


Download the json file and save it. 

Download the weights that are backed up in drive

In [0]:
%cd 
%cd ..
%cd content
!cp /content/gdrive/'My Drive'/yolov3/yolov3_custom_last.weights /darknet

Run the following command to see the recall

In [0]:
!./darknet detector recall data/obj.data cfg/yolov3_custom.cfg yolov3_custom_last.weights

Run the following command to see the mAP 

In [0]:
!./darknet detector map data/obj.data cfg/yolov3_custom.cfg yolov3_custom_last.weights

TO find the precision run the following command

In [0]:
!./darknet detector precision data/obj.data cfg/yolov3_custom.cfg yolov3_custom_last.weights

In case there is a need to calculate all the above metrics on you own and understand it . Please visit [ this link](https://towardsdatascience.com/evaluating-performance-of-an-object-detection-model-137a349c517b)

In [0]:
#Writing average , precision and recall in dict at IOU threshold 0.5
dictionary = {
    'mAP' : 0.8841,
    'precision' : 0.95,
    'recall' : 0.85
}
import json 
with open('metrics.json', 'w') as fp:
    json.dump(dictionary, fp)

Look at my google drive folder incase you need any help [Folder](https://drive.google.com/drive/folders/1xtLAYfkVWLDh0aU2O5g_XaJrboRsAU7X?usp=sharing)